In [27]:
import requests
import os
from dotenv import load_dotenv, find_dotenv
from goodreads import client
import goodreads
import codecs
from kitchen.text.converters import to_unicode
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

# TODO, fix that stupid unicode/utf-8 issue
# TODO, if author check fails, do a different search

GOODREADS_KEY    = os.environ.get('GOODREADS_KEY')
GOODREADS_SECRET = os.environ.get('GOODREADS_SECRET') 

filename = "C:\Users\dk\Documents\Projects\locus\data\locus2016.txt"
FAILED_PATH = '../results/failed.tmp.txt'
SCORED_PATH = '../results/scored.tmp.txt'

gc = client.GoodreadsClient(GOODREADS_KEY, GOODREADS_SECRET)

def rank_lookup(book):
    '''search_books(self, q, page=1, search_field='all')
     |      Get the most popular books for the given query. This will search all
     |      books in the title/author/ISBN fields and show matches, sorted by
     |      popularity on Goodreads.
     |      :param q: query text
     |      :param page: which page to return (default 1)
     |      :param search_fields: field to search, one of 'title', 'author' or
     |      'genre' (default is 'all')
    '''
    searchterm = book['title'].lstrip()
    returnvalue = gc.search_books(searchterm,page=1,search_field='title')
    return returnvalue[0].average_rating,returnvalue[0].authors,returnvalue[0].publication_date

def read_input(filename):
    with open(filename,"r") as fh:
        data = fh.readlines()
    return data

def recode(string):
    description = to_unicode(string, 'utf-8')
    return description

def parse_file(data):
    booklist = []
    for line in data:
        if "(" in line:
            title = recode(line.split(",")[0])
            author = recode(line.split(",")[1].split("(")[0])
            booklist.append({'title':title,'author':author})
    return booklist

def save_list(booklist,PATH):
    with open(PATH,'w') as outfile:
        for book in booklist:
            outfile.write(book + '\n')

def lookup_entry(booklist):
    scored_books = []; failed_books = []
    for i,book in enumerate(booklist):
        rank,checkauthor,pubdate = rank_lookup(book)
        try:
            output_string =  ','.join([str(i), rank, book['title'].lstrip(), book['author'],\
                                       str(checkauthor[0]), pubdate[2] ])
            print output_string
            scored_books.append(output_string)
        except:
            output_string =  ','.join([str(i), 'failed-rank', book['title'].lstrip(), book['author'],\
                                       'failed-author', 'failed-date'])
            print output_string
            scored_books.append(output_string)
            failed_books.append(output_string)
            
    save_list(failed_books,FAILED_PATH)
    save_list(scored_books,SCORED_PATH)
    
    return scored_books,failed_books
        
data = read_input(filename)
booklist = parse_file(data)
scored_books, failed_books = lookup_entry(booklist)

0,3.66,Company Town, Madeline Ashby ,Madeline Ashby,2016
1,3.80,The Medusa Chronicles, Stephen Baxter & Alastair Reynolds ,Stephen Baxter,2016
2,3.47,Take Back the Sky, Greg Bear ,Lucy Saxon,2014
3,3.64,Visitor, C.J. Cherryh ,Sally Beauman,2014
4,4.16,Babylon's Ashes, James S.A. Corey ,James S.A. Corey,2016
failed:  Europe in Winter
6,3.77,False Hearts, Laura Lam ,Laura Lam,2016
7,4.49,Death's End, Cixin Liu ,Liu Cixin,2016
8,3.82,The Corporation Wars: Dissidence, Ken MacLeod ,Ken MacLeod,2016
failed:  Into Everywhere
10,4.05,Faller, Will McIntosh ,Jasinda Wilder,2013
11,4.25,After Atlas, Emma Newman ,Emma Newman,2016
12,3.89,The Core of the Sun, Johanna Sinisalo ,Johanna Sinisalo,2016
13,3.41,Occupy Me, Tricia Sullivan ,Tricia Sullivan,2016
14,4.53,Rosewater, Tade Thompson ,Tade Thompson,2016
failed:  Central Station
16,3.43,Icon, Genevieve Valentine ,Margaret Stohl,2013
17,4.06,The Underground Railroad, Colson Whitehead ,Colson Whitehead,2016
18,3.17,Alien Morning, Rick Wilber ,Rick 

TypeError: unsupported operand type(s) for +: 'dict' and 'str'